In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
ratings = pd.read_csv('all_data.csv').drop('Unnamed: 0', axis=1)
avg_price = pd.read_csv('avg_prices.csv').drop('Unnamed: 0', axis=1)

In [8]:
ratings.rename(columns={'Rating': 'Customer_Rating'}, inplace=True)

Questions to ask the user: 
<br> Do you want to sit on the home side, away side, or no preference?
<br> Price range
<br> Cover or uncovered? 
<br> On a scale of 1-10, how close do you want to be to the beer garden? (10 being closest) Or no preference? 

In [11]:
change_beer = {10:1, 9:2, 8:3, 7:4, 6:5, 5:6, 4:7, 3:8, 2:9, 1:10}

ratings['Beer_Rating'] = ratings['Beer_Rating'].apply(lambda x: change_beer[x])

ratings.drop('Price', axis=1, inplace=True)

ratings.drop('No_Tickets', axis=1, inplace=True)

In [44]:
avg_price['Section'] = avg_price['Section'].astype(int)
avg_price = avg_price.groupby(['Section', 'Row']).mean().reset_index()

In [56]:
ratings['Section'] = ratings['Section'].astype(int)
ratings['Row'] = ratings['Row'].astype(str)
avg_price['Row'] = avg_price['Row'].astype(str)

In [57]:
complete = avg_price.merge(ratings, on=['Section', 'Row'], how='outer')

In [59]:
beer_rating = {10: [234, 201, 301, 101, 135, 136, 137, 138, 139, 133], 
               9: [333, 233, 302, 202, 132, 102], 
               8: [332, 232, 302, 202, 131, 103], 
               7: [331, 231, 303, 203, 130, 104], 
               6: [330, 230, 304, 204, 129, 105], 
               5: [329, 229, 305, 205, 128, 106], 
               4: [328, 228, 306, 206, 127, 107], 
               3: [327, 227, 307, 207, 126, 108], 
               2: [326, 226, 308, 208, 125, 109], 
               1: [325, 225, 309, 310, 311, 209, 210, 211, 
                   125, 224, 223, 222, 221, 220, 219, 218,
                   217, 216, 215, 214, 213, 212, 122, 121, 
                   120, 119, 118, 117, 116, 115, 115, 110]}

In [64]:
for idx, ticket in complete.iterrows():
    sec = ticket['Section']
    for key in beer_rating.keys():
        if sec in beer_rating[key]:
            complete.loc[idx, 'Beer_Rating'] = key

In [76]:
complete.drop(complete[complete['Row'] == 'Zone Seating'].index, inplace=True)

In [67]:
covered = []
covered.extend(range(201, 212))
covered.extend(range(225, 235))
covered.extend(range(301, 312))
covered.extend(range(325, 334))

In [68]:
for idx, ticket in complete.iterrows():
    if ticket['Section'] in covered:
        complete.loc[idx, 'Covered'] = 1
    else: 
        complete.loc[idx, 'Covered'] = 0

In [106]:
x = complete[complete['Customer_Rating'].astype(str) != 'nan'].drop('Customer_Rating', axis=1)

In [107]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=7, random_state=0).fit(x)

In [108]:
km.predict(complete[complete['Customer_Rating'].astype(str) == 'nan'].drop('Customer_Rating', axis=1))

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 6, 6, 6, 6, 1, 1, 1, 6, 6, 6, 3, 6, 3, 3, 3, 6, 6, 6, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 6,
       6, 6, 6, 3, 6, 3, 3, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 3, 3, 6,
       3, 3, 6, 6, 6, 6, 6, 3, 6, 6, 3, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2,
       5, 5,

In [109]:
x['Label'] = km.labels_

In [110]:
x['Customer_Rating'] = complete[complete['Customer_Rating'].astype(str) != 'nan']['Customer_Rating']

In [111]:
x.groupby('Customer_Rating').median()

,Section,Price,Beer_Rating,Covered,Label
Customer_Rating,,,,,
1.0,332,46.0000,8.0,1.0,5
2.0,325,51.5000,8.0,1.0,4
3.0,307,77.5825,7.0,1.0,2
4.0,126,121.5000,8.0,0.0,3
5.0,107,243.2500,6.0,0.0,6


In [119]:
updated_ratings = {5:1, 4:2, 2:3, 3:4, 6:5, 1:2, 0:1}

In [120]:
rest = complete[complete['Customer_Rating'].astype(str) == 'nan']
rest['Label'] = km.predict(complete[complete['Customer_Rating'].astype(str) == 'nan'].drop('Customer_Rating', axis=1))
for idx, row in rest.iterrows():
    rest.loc[idx, 'Customer_Rating'] = updated_ratings[row['Label']]

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [122]:
total = pd.concat([rest, x]).drop('Label', axis=1)

In [126]:
total[(total['Beer_Rating'] < 3) & (total['Covered'] == 1)].sort_values(['Price', 'Row'])

,Beer_Rating,Covered,Customer_Rating,Price,Row,Section
542,1.0,1.0,1.0,37.500,58,325
539,1.0,1.0,1.0,42.500,55,325
569,2.0,1.0,3.0,44.000,32,326
546,1.0,1.0,3.0,44.900,64,325
547,1.0,1.0,3.0,44.900,64,325
544,1.0,1.0,1.0,45.830,62,325
524,1.0,1.0,1.0,46.000,37,325
545,1.0,1.0,3.0,48.165,63,325
532,1.0,1.0,1.0,49.000,48,325
548,1.0,1.0,1.0,49.250,66,325


In [173]:
total.to_csv('data.csv')

In [131]:
home = []
home.extend(range(113, 140))
home.extend(range(213, 240))
home.extend(range(313, 340))

In [141]:
def answers():
    result = {}
    result['Covered'] = input('Do you want to be under cover, or no preference? (y/n/np): ').lower()
    result['Beer_Proximity'] = input('On scale of 1-3, how important is it that you be close to the beer garden or no preference? \
    (3 = Very Important, 2 = Somewhat Important, 1 = I do not care): ')
    result['Price_Max'] = input('What is the maximum price you would like to pay?: ')
    result['Price_Min'] = input('What is the minimum price you would like to pay?: ')
    result['Home_Away'] = input('Do you want to sit on the home side, away side, or no preference? (Home = 1, Away = 2, No Preference = 3): ')
    return result

ask them to give a section and row number of where they have previously sat and whether they liked it or not 

In [169]:
def filter_data(response):
    result = total.copy()
    if response['Covered'] == 'y':
        result = result[result['Covered'] == 1]
    elif response['Covered'] == 'n':
        result = result[result['Covered'] == 0]
    if response['Beer_Proximity'] == '3':
        result = result[result['Beer_Rating'] > 7]
    elif response['Beer_Proximity'] == '2':
        result = result[result['Beer_Rating'] > 4]
    result = result[(result['Price'] > int(response['Price_Min'])) & (result['Price'] < int(response['Price_Max']))]
    if response['Home_Away'] == '1':
        result = result[result['Home_Away'] == 1]
    elif response['Home_Away'] == '2':
        result = result[result['Home_Away'] == 0]
    return result[['Section', 'Row', 'Price', 'Customer_Rating']].sort_values('Customer_Rating', ascending=False)

In [171]:
response = answers()

In [172]:
filter_data(response)

,Section,Row,Price,Customer_Rating
100,109,1,148.750,5.0
67,106,2,282.875,5.0
79,107,1,243.250,5.0
78,107,1,243.250,5.0
76,106,6,245.250,5.0
98,108,8,176.000,5.0
99,108,9,152.330,5.0
50,105,3,186.875,5.0
75,106,4,250.310,5.0
72,106,3,268.375,5.0


In [148]:
total['Home_Away'] = total['Section'].apply(lambda x: 1 if x in home else 0)

include bathrooms, other data sources

In [174]:
restrooms = [110, 125, 136, 207, 305, 309]

In [199]:
total['Family_Bathrooms'] = 100
for idx, row in total.iterrows():
    sec = row['Section']
    if sec > 324:
        sec -= 200
    cur = row['Family_Bathrooms']
    for br in restrooms:
        if np.abs((sec - br)) < cur:
            total.loc[idx, 'Family_Bathrooms'] = np.abs(sec-br)
            cur = np.abs(sec-br)

In [202]:
total

,Beer_Rating,Covered,Customer_Rating,Price,Row,Section,Home_Away,Family_Bathrooms
0,9.0,0.0,4.0,117.250,1,102,0,8
1,9.0,0.0,4.0,100.000,10,102,0,8
2,9.0,0.0,4.0,99.500,11,102,0,8
3,9.0,0.0,4.0,112.830,12,102,0,8
4,9.0,0.0,4.0,113.860,2,102,0,8
5,9.0,0.0,4.0,86.000,3,102,0,8
8,9.0,0.0,4.0,96.000,6,102,0,8
9,9.0,0.0,4.0,101.670,7,102,0,8
10,9.0,0.0,4.0,97.670,8,102,0,8
11,9.0,0.0,4.0,105.670,9,102,0,8


In [204]:
total['Beer_Rating'] = total['Beer_Rating'].apply(lambda x: np.abs(x-11))

In [205]:
total.rename(columns={'Beer_Rating': 'Proximity_to_Beer'}, inplace=True)

In [207]:
total.to_csv('data.csv')